In [1]:
# Python version
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
import pickle as pickle
from sklearn.datasets import make_classification

Python: 3.6.4 |Anaconda custom (64-bit)| (default, Jan 16 2018, 18:10:19) 
[GCC 7.2.0]
scipy: 1.0.1
numpy: 1.14.2
matplotlib: 2.1.2
pandas: 0.22.0
sklearn: 0.19.1


In [49]:
from sklearn import model_selection,linear_model,metrics
from sklearn.metrics import accuracy_score,log_loss
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,roc_curve

In [3]:
def pickle_data(filename, data):

    fileObject = open('PickleData/'+filename,'wb') 
    # this writes the object a to the
    # file named 'testfile'
    pickle.dump(data,fileObject) 

    # here we close the fileObject
    fileObject.close()
    
def load_pickle_data(filename):
    # we open the file for reading
    fileObject = open('PickleData/'+filename,'rb')  
    # load the object from the file into var b
    return pickle.load(fileObject)

## Load Pickle Data

#### Binary DF

In [4]:
X = load_pickle_data('binary_x_train')
y = load_pickle_data('binary_y_train')

In [5]:
X_val = load_pickle_data('binary_x_val')
y_val = load_pickle_data('binary_y_val')

#### Non-Binary DF

In [36]:
df = load_pickle_data('normal_clean')
X = df.drop(['click'],axis=1)
y = df['click']

In [11]:
df = load_pickle_data('val_normal_clean')
X_val = df.drop(['click'],axis=1)
y_val = df['click']

#### Undersampled Data

In [6]:
#Pickle binary dataframe
X_undersampled = load_pickle_data('binary_x_train_undersampled')
y_undersampled = load_pickle_data('binary_y_train_undersampled')


## Run Logistic Regression

In [44]:
#  Logistic regression with parameters set
logisticRegr = LogisticRegression(multi_class='multinomial',solver ='newton-cg',max_iter=500)

In [45]:
logisticRegr.fit(X_undersampled, y_undersampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=0.0001, verbose=0, warm_start=False)

In [46]:
pred_y = logisticRegr.predict_proba(X_val)[:,1]

In [222]:
#Pickle data to use for bid estimation
pickle_data('Pred_y_logistic_binary', pred_y)

In [47]:
classification_threshold = 0.5
bin_pred_y = (pred_y>classification_threshold)*1

##### Confusion Matrix

In [50]:
confusion_matrix = pd.DataFrame(
    confusion_matrix(y_val,bin_pred_y), 
    columns=["Predicted False", "Predicted True"], 
    index=["Actual False", "Actual True"]
)
display(confusion_matrix)

,Predicted False,Predicted True
Actual False,236568,67155
Actual True,58,144


##### Classifical Report

In [15]:
print (metrics.classification_report(y_val, bin_pred_y))

             precision    recall  f1-score   support

          0       1.00      0.78      0.88    303723
          1       0.00      0.71      0.00       202

avg / total       1.00      0.78      0.88    303925



##### AUC

In [16]:
fpr, tpr, thresholds = metrics.roc_curve(y_val.values, bin_pred_y, pos_label=1)
print('AUC:',round(metrics.auc(fpr, tpr),5))

AUC: 0.74588


##### RMSE

In [17]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_val, bin_pred_y))
print('RMSE:',rms)

RMSE: 0.4702658341391338


##### Log Loss

In [18]:
print('Log Loss:',log_loss(y_val, bin_pred_y))

Log Loss: 7.63842551549414


## Run Linear Regression

In [30]:
LinearRegr = LinearRegression(normalize=True)

In [31]:
LinearRegr.fit(X_undersampled, y_undersampled)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [34]:
pred_y = LinearRegr.predict(X_val)

##### Apply logistic signmoid and threshold

In [35]:
#Run output through logistic sigmoid function to get values between 0-1
#to create a mapping between a scale with range [0,1] and range [−∞,+∞]
log_pred_y = logistic.cdf(pred_y)

In [36]:
#Set threshold of classification, 0 or 1
classification_threshold = 0.5
bin_pred_y = (pred_y>classification_threshold)*1

##### Confusion Matrix 

In [39]:
confusion_matrix = pd.DataFrame(
    confusion_matrix(y_val,bin_pred_y), 
    columns=["Predicted False", "Predicted True"], 
    index=["Actual False", "Actual True"]
)
display(confusion_matrix)

,Predicted False,Predicted True
Actual False,233578,70145
Actual True,61,141



##### Classification Report

In [40]:
print (metrics.classification_report(y_val, bin_pred_y))
# print (metrics.accuracy_score(y, pred_y))

             precision    recall  f1-score   support

          0       1.00      0.77      0.87    303723
          1       0.00      0.70      0.00       202

avg / total       1.00      0.77      0.87    303925



##### AUC

In [42]:
fpr, tpr, thresholds = metrics.roc_curve(y_val, bin_pred_y, pos_label=1)
print('AUC:',round(metrics.auc(fpr, tpr),5))

AUC: 0.73353


##### RMSE

In [43]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_val, bin_pred_y))
print('RMSE:',rms)

RMSE: 0.4806222831468941


##### Log Loss

In [ ]:
print('Log Loss:',log_loss(y_val, bin_pred_y))